# EnergyPlus and TRNSYS Example

In [1]:
import os
from zerobnl import CoSim

You can safely ignore the following error (it will also be in the nodes logs):

RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88

-> [Numpy documentation](https://github.com/numpy/numpy/pull/432)

In [2]:
sim = CoSim()

# There are 4 environments in this example.

sim.create_meta_model("MetaPLANT", [("QDH", "W"),("COGw_Flag","-")], [("Tsupply", "Cdeg"),("MDOTsupply", "kgs")]) # Set, Get
sim.create_environment("EnvPLANT", "wrapper_plant.py", "Dockerfile_na")

sim.create_meta_model("MetaDEM", [("Tthermostat", "Cdeg")], [("QDH", "W"),("Tindoor","Cdeg")]) # Set, Get
sim.create_environment("EnvDEM", "wrapper_dem.py", "Dockerfile_na")

sim.create_meta_model("MetaCTRL", [("demandOK", "-")], [("COGwflag", "-")]) # Set, Get
sim.create_environment("EnvCTRL", "wrapper_ctrl.py", "Dockerfile_ctrl")

sim.create_meta_model("MetaLCTRL", [("Tindoor", "Cdeg"),("mdotTOT", "kgs")], [("Tth", "Cdeg"),("demandFlag", "-")]) # Set, Get
sim.create_environment("EnvLCTRL", "wrapper_lctrl.py", "Dockerfile_lctrl")

# There is one model per each environment in this example. 
# OBS!! The name Base# indicates a model. Each model is contained in a node. There can be more nodes in each environment.
sim.add_node("Base0", "MetaLCTRL", "EnvLCTRL", init_values={},local=False)
sim.add_node("Base1", "MetaCTRL", "EnvCTRL", init_values={},local=False)
sim.add_node("Base2", "MetaPLANT", "EnvPLANT", init_values={"QDH":2000,"COGw_Flag":1}, files=["COGplant.fmu"],local=True)
sim.add_node("Base3", "MetaDEM", "EnvDEM", init_values={}, files=["Residential_DH.fmu"],local=True)


# Links among nodes
sim.add_link("Base0", "Tth", "Base3", "Tthermostat") # get from Base0 and set to Base1
sim.add_link("Base0", "demandFlag", "Base1", "demandOK") # get from Base0 and set to Base1
sim.add_link("Base1", "COGwflag", "Base2", "COGw_Flag") # get from Base1 and set to Base0
sim.add_link("Base2", "MDOTsupply", "Base0", "mdotTOT") # get from Base1 and set to Base0
sim.add_link("Base3", "QDH", "Base2", "QDH") # get from Base1 and set to Base0
sim.add_link("Base3", "Tindoor", "Base0", "Tindoor") # get from Base1 and set to Base0

# Create groups from the simulation sequence. Nodes in the same group run in parallel. 
# A group is defined within the first level of square brackets.
sim.create_sequence([["Base0"], ["Base1"],["Base2"],["Base3"]])
sim.set_time_unit("seconds")
sim.create_steps([3600] * 24)

Once the next step has been launched, logging `INFO :: Waiting for local nodes to run..`, you need to run tho following command `wrapper_eplus.py Base1 GRP1` in the indicated folder (in a dedicated environment) in order to run the local node.

In [3]:
sim.run()

INFO :: Waiting for local nodes to run...
INFO :: Simulation finished in 0 min and 38 sec


If you see `INFO :: Simulation finished in X min and Y sec` it means everything went well.
You can find logs of the nodes in the file `nodes.log`, it's a text file you can open it directly in Jupyter or in your favorite text editor.

At the begining of the file you will find a serie of:

`Step X/10 : DO SOMETHING
 ---> 29d2f3226daf`
 
It's the logs of the creation of the Docker image, based on the provided Dockerfile (here `Dockerfile_base`).

Then all the logs are structures in the same way:

`<node>    | <level> :: <message>`

* `node` refers to the concerned simulation node or orchestrator
* `level` can be `DEBUG`: used for development purpose, `INFO`: giving you info on the running process, `WARNING`: warning you on action to make or some weird behaviour, `ERROR`: something went wrong and `CRITICAL`: something went really wrong.
* `message` is the body of the log, it describes what's happening.

You can also find information on the ongoing simulation in the file `activity.log` (in the root folder for the main processus and on the temporary folder for each node)

In [4]:
sim.connect_to_results_db()
sim.get_list_of_available_results()

,IN/OUT,Node,Attribute
0,IN,Base0,Tindoor
1,IN,Base2,COGw_Flag
2,IN,Base1,demandOK
3,OUT,Base2,MDOTsupply
4,OUT,Base3,QDH
5,IN,Base0,mdotTOT
6,IN,Base3,Tthermostat
7,IN,Base2,QDH
8,OUT,Base1,COGwflag
9,OUT,Base3,Tindoor


The name to the stored results are build as `<type>||<node>||<attribute>`.

`type` can be:
* `IN` if it's an input attribute (to set - stored automatically)
* `OUT` if it's an output attribute (to get - stored automatically)
* `X` if it's an internal value (stored by the user, using the `save_attribute()` method in the wrapper)

Knowing this, you can create matching pattern using `*` in order to properly select results.

In [5]:
for key, value in sim.get_results_by_pattern("OUT*Base0*").items():
    print(key)
    print(value)   

INFO :: Matching results: Base0 - demandFlag
INFO :: Matching results: Base0 - Tth


OUT||Base0||Tth
2000-01-01 01:00:00    20.0
2000-01-01 02:00:00    20.0
2000-01-01 03:00:00    20.0
2000-01-01 04:00:00    20.0
2000-01-01 05:00:00    20.0
2000-01-01 06:00:00    20.0
2000-01-01 07:00:00    20.0
2000-01-01 08:00:00    20.0
2000-01-01 09:00:00    20.0
2000-01-01 10:00:00    20.0
2000-01-01 11:00:00    20.0
2000-01-01 12:00:00    20.0
2000-01-01 13:00:00    20.0
2000-01-01 14:00:00    20.0
2000-01-01 15:00:00    20.0
2000-01-01 16:00:00    20.0
2000-01-01 17:00:00    20.0
2000-01-01 18:00:00    20.0
2000-01-01 19:00:00    20.0
2000-01-01 20:00:00    20.0
2000-01-01 21:00:00    20.0
2000-01-01 22:00:00    20.0
2000-01-01 23:00:00    20.0
2000-01-02 00:00:00    20.0
dtype: float64
OUT||Base0||demandFlag
2000-01-01 01:00:00    0.0
2000-01-01 02:00:00    0.0
2000-01-01 03:00:00    0.0
2000-01-01 04:00:00    0.0
2000-01-01 05:00:00    0.0
2000-01-01 06:00:00    0.0
2000-01-01 07:00:00    0.0
2000-01-01 08:00:00    0.0
2000-01-01 09:00:00    0.0
2000-01-01 10:00:00    0.0
2000

In [6]:
import matplotlib.pyplot as plt
for key, value in sim.get_results_by_pattern("OUT*Base1*").items():
    plt.figure(figsize=(18, 8))
    plt.plot(value, "o-", label="Treturn_DH", alpha=1)
    plt.legend()

INFO :: Matching results: Base1 - COGwflag


In [7]:
for key, value in sim.get_results_by_pattern("IN*Base0*").items():
    print(key)
    print(value)

INFO :: Matching results: Base0 - Tindoor
INFO :: Matching results: Base0 - mdotTOT


IN||Base0||Tindoor
2000-01-01 01:00:00    20.345433
2000-01-01 02:00:00    20.345433
2000-01-01 02:00:00    20.345433
2000-01-01 02:00:00    20.345433
2000-01-01 02:00:00    20.320743
2000-01-01 03:00:00    20.320743
2000-01-01 03:00:00    20.320743
2000-01-01 03:00:00    20.320743
2000-01-01 03:00:00    20.304796
2000-01-01 04:00:00    20.304796
2000-01-01 04:00:00    20.304796
2000-01-01 04:00:00    20.304796
2000-01-01 04:00:00    20.295240
2000-01-01 05:00:00    20.295240
2000-01-01 05:00:00    20.295240
2000-01-01 05:00:00    20.295240
2000-01-01 05:00:00    20.285695
2000-01-01 06:00:00    20.285695
2000-01-01 06:00:00    20.285695
2000-01-01 06:00:00    20.285695
2000-01-01 06:00:00    20.280745
2000-01-01 07:00:00    20.280745
2000-01-01 07:00:00    20.280745
2000-01-01 07:00:00    20.280745
2000-01-01 07:00:00    20.276751
2000-01-01 08:00:00    20.276751
2000-01-01 08:00:00    20.276751
2000-01-01 08:00:00    20.276751
2000-01-01 08:00:00    20.287851
2000-01-01 09:00:00    2

In [8]:
for key, value in sim.get_results_by_pattern("IN*Base1*").items():
    print(key)
    print(value)

INFO :: Matching results: Base1 - demandOK


IN||Base1||demandOK
2000-01-01 00:00:00    0.0
2000-01-01 01:00:00    0.0
2000-01-01 01:00:00    0.0
2000-01-01 01:00:00    0.0
2000-01-01 01:00:00    0.0
2000-01-01 02:00:00    0.0
2000-01-01 02:00:00    0.0
2000-01-01 02:00:00    0.0
2000-01-01 02:00:00    0.0
2000-01-01 03:00:00    0.0
2000-01-01 03:00:00    0.0
2000-01-01 03:00:00    0.0
2000-01-01 03:00:00    0.0
2000-01-01 04:00:00    0.0
2000-01-01 04:00:00    0.0
2000-01-01 04:00:00    0.0
2000-01-01 04:00:00    0.0
2000-01-01 05:00:00    0.0
2000-01-01 05:00:00    0.0
2000-01-01 05:00:00    0.0
2000-01-01 05:00:00    0.0
2000-01-01 06:00:00    0.0
2000-01-01 06:00:00    0.0
2000-01-01 06:00:00    0.0
2000-01-01 06:00:00    0.0
2000-01-01 07:00:00    0.0
2000-01-01 07:00:00    0.0
2000-01-01 07:00:00    0.0
2000-01-01 07:00:00    0.0
2000-01-01 08:00:00    0.0
                      ... 
2000-01-01 17:00:00    0.0
2000-01-01 17:00:00    0.0
2000-01-01 17:00:00    0.0
2000-01-01 17:00:00    0.0
2000-01-01 18:00:00    0.0
2000-01-

In [9]:
for key, value in sim.get_results_by_pattern("OUT*Base1*").items():
    print(key)
    print(value)

INFO :: Matching results: Base1 - COGwflag


OUT||Base1||COGwflag
2000-01-01 01:00:00    1.0
2000-01-01 02:00:00    1.0
2000-01-01 03:00:00    1.0
2000-01-01 04:00:00    1.0
2000-01-01 05:00:00    1.0
2000-01-01 06:00:00    1.0
2000-01-01 07:00:00    1.0
2000-01-01 08:00:00    1.0
2000-01-01 09:00:00    1.0
2000-01-01 10:00:00    1.0
2000-01-01 11:00:00    1.0
2000-01-01 12:00:00    1.0
2000-01-01 13:00:00    1.0
2000-01-01 14:00:00    1.0
2000-01-01 15:00:00    1.0
2000-01-01 16:00:00    1.0
2000-01-01 17:00:00    1.0
2000-01-01 18:00:00    1.0
2000-01-01 19:00:00    1.0
2000-01-01 20:00:00    1.0
2000-01-01 21:00:00    1.0
2000-01-01 22:00:00    1.0
2000-01-01 23:00:00    1.0
2000-01-02 00:00:00    1.0
dtype: float64


In [10]:
for key, value in sim.get_results_by_pattern("OUT*Base3*").items():
    print(key)
    print(value)

INFO :: Matching results: Base3 - QDH
INFO :: Matching results: Base3 - Tindoor


OUT||Base3||QDH
2000-01-01 01:00:00    1154.799177
2000-01-01 02:00:00    1070.470865
2000-01-01 03:00:00    1007.785085
2000-01-01 04:00:00     965.593421
2000-01-01 05:00:00     937.545649
2000-01-01 06:00:00     919.468500
2000-01-01 07:00:00     913.565288
2000-01-01 08:00:00     953.785361
2000-01-01 09:00:00    1001.530066
2000-01-01 10:00:00    1020.704683
2000-01-01 11:00:00     929.414054
2000-01-01 12:00:00     864.677355
2000-01-01 13:00:00     834.337441
2000-01-01 14:00:00     841.296775
2000-01-01 15:00:00    1039.836157
2000-01-01 16:00:00    1177.260391
2000-01-01 17:00:00    1226.097943
2000-01-01 18:00:00    1249.043278
2000-01-01 19:00:00    1266.247812
2000-01-01 20:00:00    1265.575800
2000-01-01 21:00:00    1264.071790
2000-01-01 22:00:00    1261.512152
2000-01-01 23:00:00    1261.757559
2000-01-02 00:00:00    1268.576308
dtype: float64
OUT||Base3||Tindoor
2000-01-01 01:00:00    20.345433
2000-01-01 02:00:00    20.320743
2000-01-01 03:00:00    20.304796
2000-01-01

In [11]:
for key, value in sim.get_results_by_pattern("OUT*Base2*").items():
    print(key)
    print(value)

INFO :: Matching results: Base2 - MDOTsupply
INFO :: Matching results: Base2 - Tsupply


OUT||Base2||MDOTsupply
2000-01-01 01:00:00    0.019934
2000-01-01 02:00:00    0.011515
2000-01-01 03:00:00    0.010674
2000-01-01 04:00:00    0.010050
2000-01-01 05:00:00    0.009629
2000-01-01 06:00:00    0.009350
2000-01-01 07:00:00    0.009169
2000-01-01 08:00:00    0.009110
2000-01-01 09:00:00    0.009512
2000-01-01 10:00:00    0.009988
2000-01-01 11:00:00    0.010179
2000-01-01 12:00:00    0.009268
2000-01-01 13:00:00    0.008623
2000-01-01 14:00:00    0.008320
2000-01-01 15:00:00    0.008390
2000-01-01 16:00:00    0.010370
2000-01-01 17:00:00    0.011740
2000-01-01 18:00:00    0.012227
2000-01-01 19:00:00    0.012456
2000-01-01 20:00:00    0.012628
2000-01-01 21:00:00    0.012621
2000-01-01 22:00:00    0.012606
2000-01-01 23:00:00    0.012580
2000-01-02 00:00:00    0.012583
dtype: float64
OUT||Base2||Tsupply
2000-01-01 01:00:00    60.0
2000-01-01 02:00:00    60.0
2000-01-01 03:00:00    60.0
2000-01-01 04:00:00    60.0
2000-01-01 05:00:00    60.0
2000-01-01 06:00:00    60.0
2000-0